In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Functions import *

# Load the data
data = pd.read_csv('Data/merged_data.csv')

max_missing_percentage = 20

columns_to_drop = []

for i in range(len(data.columns)):
    missing_count = data.iloc[:, i].isnull().sum()
    missing_percentage = missing_count / len(data) * 100
    
    # Add column to drop list if missing_percentage is greater than max_missing_percentage
    if missing_percentage > max_missing_percentage:
        columns_to_drop.append(data.columns[i])

# Drop the columns after iterating
data = data.drop(columns_to_drop, axis=1)

#Calculate mean of all variables starting with "sun"
sun_columns = data.filter(like='sun_')
sun_mean = sun_columns.mean(axis=1)
data['sun_mean'] = sun_mean
data['sun_mean'] = data['sun_mean'].fillna(0)

for column in data.columns:
    # Check if the column starts with 'sun_' and is not 'sun_mean'
    if column.startswith('sun_') and column != 'sun_mean':
        for row in range(len(data)):
            # Check if the value is NaN
            if np.isnan(data.loc[row, column]):
                # Replace NaN with the mean of the corresponding row
                data.loc[row, column] = data['sun_mean'].loc[row]

# Calculate mean of all variables starting with "wind"
wind_columns = data.filter(like='wind_')
wind_mean = wind_columns.mean(axis=1)
data['wind_mean'] = wind_mean
data['wind_mean'] = data['wind_mean'].fillna(0)

for column in data.columns:
    # Check if the column starts with 'wind_' and is not 'wind_mean'
    if column.startswith('wind_') and column != 'wind_mean':
        for row in range(len(data)):
            # Check if the value is NaN
            if np.isnan(data.loc[row, column]):
                # Replace NaN with the mean of the corresponding row
                data.loc[row, column] = data['wind_mean'].loc[row]

area_to_drop = 'DK1_spot'

# Drop DK1_spot and DATE
data = data.drop([area_to_drop, 'DATE'], axis=1)

# data = data.drop(['sun_mean', 'wind_mean'], axis=1)

data_np = data.to_numpy()

y = data_np[:, 0].reshape(-1, 1)
weather = data_np[:, 1:-2]


In [ ]:
# Save data
data.to_csv('Data/non_nan_data.csv', index=False)

Standard AR-model with exogenous variables

In [5]:
p = 1
con = 1
tr = 0
# exog = weather
exog = 0

Beta, SEbeta, _, Pvalue, tratioBeta, _, _, _, aiccrit, hqccrit, siccrit = VARlsExog(y, p, con, tr, exog)

# Collect beta, SEbeta and tratioBeta in a single dataframe
results = pd.DataFrame()
results['Beta'] = Beta.reshape(-1, order='F')
results['SEbeta'] = SEbeta
results['Pvalue'] = Pvalue

results.round({'Beta': 3, 'SEbeta': 3, 'Pvalue': 3})

print(aiccrit, hqccrit, siccrit)

results

9.416509331212024 9.416570758821651 9.416711145121218


,Beta,SEbeta,Pvalue
0,0.980455,0.000643,0.0
1,8.938638,0.466599,0.0
